### Implementation of non-trainable attention weights 

Chapter 3 of Seb Raschka LLM book

In [1]:
import torch 

In [ ]:
# input words (each one is tokenized and now embedded)
inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

6

**Computing Attention Score for each input word/token**  

Attention score is calculated through dot product between other words  
-  Each attention weight index is calculated by multiplying the correpsonding input with our current word 
-  We can see the attention weight as a correlation/similarity between the current word with other words 
-  Computing this for all inputs can be formed as a NxN matrix where N is the total number of input, that compares the attention score with each other 

In [5]:
# calculate attention block for the second word 
query= inputs[1]    # get the input word journey 
attn_score_2= torch.empty(inputs.shape[0])  # attention blocks for each of the input word 
for i, x_i in enumerate (inputs):
    attn_score_2[i] = torch.dot(x_i, query) # dot product of each word with our attention block word 
print(attn_score_2)

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


Normlize attention score to get actual attention weights so that all attention **adds up to be 1**   

This method of normalization is as calculating probability/percentage. Softmax might be better at handling extreme (high & low) values 

In [ ]:
attn_weight_temp_2= attn_score_2/attn_score_2.sum() 
print(attn_weight_temp_2)   # normalized attention weights 

tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])


In [ ]:
def softmax(x):     # we can use PyTorch implemented softmax 
    return torch.exp(x)/ (torch.exp(x)).sum(dim=0)
attn_weight_softmax_2= softmax(attn_score_2)
print(attn_weight_softmax_2)    # normalized with softmax 

tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])


**Calculate attention weights : ( normalized attention_score  * each input word/token ) sum up for all inputs**

In [ ]:
query = inputs[1] 
context_vec_2= torch.zeros(query.shape) # same shaped attention weights 
for i, x_i in enumerate (inputs):
    context_vec_2 += attn_weight_softmax_2[i] * x_i     # multiply and element wise add
print (context_vec_2)   # attention block for second input 


tensor([0.4419, 0.6515, 0.5683])


**Generalize to all inputs**

In [ ]:
attention_score= torch.empty(6,6)   # 6 inputs and 1 attention score for each 
for i, x_i in enumerate(inputs):
    for j, x_j in enumerate(inputs):
        attention_score[i,j] = torch.dot(x_j, x_i)  # dot product fo similarity 
print (attention_score) # attention score table 

''' 
Alternative faster 
attention_score= inputs @ inputs.T  # matrix multiplication 
'''

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [16]:
# Now we need to normalize 
attention_weights= torch.softmax(attention_score, dim=-1)
print(attention_weights)

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])


In [ ]:
# multiply with the attention weights to get attention block 
all_context_block= attention_weights @ inputs # 6x6  X 6x3 
# matrix mult: multiply each x_i with corresponding entry (we sum up the attention weights, so we can use matrix multiplication)
print(all_context_block)

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])
